#### Prerequesites:
* Define schema (make figure) 
* Define structure (make figure)
* Define schema class in python (src/models/model_schemata.py)
* Convert schema + structure into model template (models/templates/p_model.pl)

#### Procedure:
* Load data
* Create lfi script
* Add uncertain evidence to lfi model as probabilistic clauses

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Importing libraries
import pandas as pd
import os

from src.models import model_schemata as schema
from src.models import build_model as build

In [3]:
# os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_pi/')
os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_pi/')

In [4]:
# reading csv from files into dict
data = {}
data['e_ksea'] = pd.read_csv('data/processed/synthetic_data/toy_network_mini/p_model/e_ksea.csv')
data['p_fc'] = pd.read_csv('data/processed/synthetic_data/toy_network_mini/p_model/p_fc.csv')

In [5]:
data['p_fc']

,phosphosite,sample,value,prob,p_dec,p_base,p_inc
0,p0,s1,base,0.534836,0.238082,0.522836,0.238082
1,p0,s1,dec,0.534836,0.522836,0.467164,0.001000
2,p0,s1,inc,0.534836,0.001000,0.467164,0.522836
3,p0,s10,base,0.886022,0.062489,0.874022,0.062489
4,p0,s10,dec,0.886022,0.874022,0.115978,0.001000
...,...,...,...,...,...,...,...
295,p0,s98,dec,0.887979,0.875979,0.114021,0.001000
296,p0,s98,inc,0.887979,0.001000,0.114021,0.875979
297,p0,s99,base,0.723788,0.143606,0.711788,0.143606
298,p0,s99,dec,0.723788,0.711788,0.278212,0.001000


In [6]:
# Mapping data to Problog predicates
predicates = {}
predicates['e_ksea'] = schema.EKseaPredicate()
predicates['e_ksea'].add_data(data['e_ksea'], 'enzyme', 'sample', 'value')
predicates['e_ksea'].add_ad_probabilities(data['e_ksea'], 'p_dec', 'p_base', 'p_inc')
predicates['p_fc'] = schema.PFoldChangePredicate()
predicates['p_fc'].add_data(data['p_fc'], 'phosphosite', 'sample', 'value')
predicates['p_fc'].add_ad_probabilities(data['p_fc'], 'p_dec', 'p_base', 'p_inc')

In [7]:
model_lfi = 'models/synthetic_data/toy_network_mini/p_model/p_model_lfi_o1.pl'
for predicate in predicates: 
    ad_generator = build.ProblogStatementGenerator(predicates[predicate])
    clauses = ad_generator.generate_ad_clauses()
    build.insert_statements(model=model_lfi, statements=clauses, location='%% p1::{}'.format(predicate)) # insert into Problog file